## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:

Please set up AWS SES (Simple Email Service) to send emails:

1. Go to the AWS Console and navigate to Amazon SES
2. Verify your sender email address (or domain) in SES
3. If you're in the SES sandbox, you'll also need to verify recipient email addresses
4. Make sure your AWS credentials are configured in your .env file with:
   - `AWS_ACCESS_KEY_ID=xxxx`
   - `AWS_SECRET_ACCESS_KEY=xxxx`
   - `AWS_REGION=us-east-1` (or your preferred region)

Note: The boto3 library will automatically use these credentials from your environment or AWS credentials file.

In [9]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import boto3
import os
from botocore.exceptions import ClientError
import asyncio

In [51]:
load_dotenv(override=True)

True

In [14]:
# Let's just check emails are working for you

def send_test_email():
    # Debug: Check if credentials are loaded
    print("Checking AWS credentials...")
    
    # Get credentials and clean them (remove whitespace and quotes)
    access_key = os.environ.get('AWS_ACCESS_KEY_ID', '').strip().strip('"').strip("'")
    secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY', '').strip().strip('"').strip("'")
    session_token = os.environ.get('AWS_SESSION_TOKEN', '').strip().strip('"').strip("'")
    region = os.environ.get('AWS_REGION', 'us-east-1').strip().strip('"').strip("'")
    
    if access_key:
        print(f"AWS_ACCESS_KEY_ID: {access_key[:5]}...{access_key[-5:]}")
    else:
        print("AWS_ACCESS_KEY_ID: Not set")
    
    if secret_key:
        print(f"AWS_SECRET_ACCESS_KEY: {secret_key[:5]}...{secret_key[-5:]}")
    else:
        print("AWS_SECRET_ACCESS_KEY: Not set")
    
    if session_token:
        print(f"AWS_SESSION_TOKEN: {session_token[:5]}...{session_token[-5:]}")
    else:
        print("AWS_SESSION_TOKEN: Not set")
    
    print(f"AWS_REGION: {region}\n")
    
    # Create SES client with cleaned credentials
    try:
        # Manually pass cleaned credentials to ensure they're used
        ses_client = boto3.client(
            'ses',
            region_name=region,
            aws_access_key_id=access_key if access_key else None,
            aws_secret_access_key=secret_key if secret_key else None,
            aws_session_token=session_token if session_token else None
        )
        
        response = ses_client.send_email(
            Source="greg.jacobs@thrivent.com",  # Change to your verified sender
            Destination={
                'ToAddresses': ["greg.jacobs@thrivent.com"]  # Change to your recipient
            },
            Message={
                'Subject': {
                    'Data': "Test email",
                    'Charset': 'UTF-8'
                },
                'Body': {
                    'Text': {
                        'Data': "This is an important test email from Python!",
                        'Charset': 'UTF-8'
                    }
                }
            }
        )
        print(f"✓ Email sent successfully! Message ID: {response['MessageId']}")
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"✗ Error sending email ({error_code}): {error_message}")
        
        if error_code == 'SignatureDoesNotMatch':
            print("\nTroubleshooting tips:")
            print("1. Check your .env file for extra quotes or spaces around values")
            print("2. Values should be: AWS_ACCESS_KEY_ID=AKIA... (no quotes)")
            print("3. Reload with load_dotenv(override=True)")
    except Exception as e:
        print(f"✗ Unexpected error: {str(e)}")

send_test_email()

Checking AWS credentials...
AWS_ACCESS_KEY_ID: ASIAS...4VELW
AWS_SECRET_ACCESS_KEY: YOECO...yPSeN
AWS_SESSION_TOKEN: IQoJb...wiCPd
AWS_REGION: us-east-2

✓ Email sent successfully! Message ID: 010f019c3941d3ec-bf21f14d-5afe-44c8-8ddf-db77b55bf009-000000


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [15]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [16]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0"
)

In [17]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Certainly! Below is a professional and serious cold email for promoting ComplAI's SaaS tool for SOC2 compliance and audit preparation.

---

**Subject: Ensure SOC2 Compliance with Ease – Discover ComplAI’s AI-Powered Solution**

**Dear [Recipient's Name],**

I hope this email finds you well. My name is [Your Name], and I am reaching out to you on behalf of ComplAI, a pioneering company dedicated to simplifying SOC2 compliance and audit preparation through cutting-edge AI technology.

In today’s rapidly evolving digital landscape, maintaining SOC2 compliance is not just a regulatory requirement but a critical component of building and maintaining customer trust. However, the complexity and manual effort involved in achieving and sustaining compliance can be daunting.

**Introducing ComplAI: Your Partner in SOC2 Compliance**

ComplAI offers a sophisticated SaaS tool designed to automate and streamline the SOC2 compliance process. Powered by advanced AI, our solution provides:

- **Automa

OPENAI_API_KEY is not set, skipping trace export


 and processes to ensure ongoing compliance.
- **Smart Audit Preparation:** Generate audit-ready documentation and reports with minimal manual intervention.
- **Risk Assessment & Mitigation:** Identify potential risks and receive actionable insights to mitigate them effectively.
- **Real-Time Alerts:** Receive immediate notifications of any compliance deviations or audit triggers.

**Why Choose ComplAI?**

- **Efficiency:** Reduce the time and resources required for compliance efforts by up to 50%.
- **Accuracy:** Leverage AI to minimize human error and ensure precise compliance documentation.
- **Scalability:** Easily scale your compliance efforts as your business grows.
- **Expert Support:** Benefit from our team of SOC2 experts who are available to assist you every step of the way.

**Take the Next Step**

I would love to discuss how ComplAI can help your organization achieve and maintain SOC2 compliance with greater efficiency and confidence. Are you available for a brief call next

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is no

In [18]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Certainly! Below is a professional and serious cold sales email for ComplAI's SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.

---

**Subject: Ensure SOC2 Compliance Effortlessly with ComplAI’s AI-Powered Solution**

**Dear [Recipient’s Name],**

I hope this email finds you well. My name is [Your Name], and I am reaching out to you on behalf of ComplAI, a leading provider of innovative SaaS solutions designed to streamline compliance processes.

In today’s rapidly evolving business landscape, maintaining SOC2 compliance is not just a necessity—it’s a competitive advantage. Non-compliance can lead to significant risks, including financial penalties, loss of customer trust, and reputational damage. 

At ComplAI, we understand the challenges that come with preparing for SOC2 audits. That’s why we have developed a cutting-edge AI-powered tool specifically designed to ensure your organization remains SOC2 compliant with minimal effort.

**Here’s what ComplAI 

In [21]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="litellm/bedrock/us.amazon.nova-pro-v1:0"
)

In [22]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: 🛡️ Ready to Ace Your Next SOC2 Audit? ComplAI Has Your Back! 🚀

Hey [Prospect's Name],

Hope this email finds you well and ready to conquer the world—or at least pass that pesky SOC2 audit with flying colors! 🌟

Picture this: It’s audit season. Your team is scrambling, spreadsheets are flying everywhere, and stress levels are through the roof. Sound familiar? We’ve all been there. But what if I told you there’s a smarter, more efficient way to ensure SOC2 compliance?

Introducing **ComplAI**—your new best friend in the world of compliance. Powered by AI, our SaaS tool is designed to make SOC2 compliance a breeze. Here’s how we can help you:

🤖 **AI-Powered Automation**: Say goodbye to manual data entry and hello to automated workflows that keep your compliance documentation up-to-date.
📊 **Real-Time Monitoring**: Get instant alerts and insights into your compliance status, so you’re always one step ahead.
📄 **Easy Audit Prep**: Our tool generates comprehensiv

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [23]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="litellm/bedrock/us.amazon.nova-pro-v1:0",
)

In [24]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='litellm/bedrock/us.amazon.nova-pro-v1:0', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [36]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """

    # Get credentials and clean them (remove whitespace and quotes)
    access_key = os.environ.get('AWS_ACCESS_KEY_ID', '').strip().strip('"').strip("'")
    secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY', '').strip().strip('"').strip("'")
    session_token = os.environ.get('AWS_SESSION_TOKEN', '').strip().strip('"').strip("'")
    region = os.environ.get('AWS_REGION', 'us-east-1').strip().strip('"').strip("'")
    
    # Create SES client with cleaned credentials
    ses_client = boto3.client(
            'ses',
            region_name=region,
            aws_access_key_id=access_key if access_key else None,
            aws_secret_access_key=secret_key if secret_key else None,
            aws_session_token=session_token if session_token else None
        )
    
    try:
        response = ses_client.send_email(
            Source="greg.jacobs@thrivent.com",
            Destination={'ToAddresses': ["greg.jacobs@thrivent.com"]},
            Message={
                'Subject': {'Data': "Sales email", 'Charset': 'UTF-8'},
                'Body': {'Text': {'Data': body, 'Charset': 'UTF-8'}}
            }
        )
        return {"status": "success", "message_id": response['MessageId']}
    except ClientError as e:
        return {"status": "error", "message": str(e)}

### This has automatically been converted into a tool, with the boilerplate json created

In [37]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10eaf7ba0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### And you can also convert an Agent into a tool

In [38]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114a8fc40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [39]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114a8fa60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ecd6520>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='Write 

## And now it's time for our Sales Manager - our planning agent

In [40]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="litellm/bedrock/us.amazon.nova-pro-v1:0")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

In [41]:
print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    The best cold sales email has been successfully sent to the CEO. If you need any further assistance or have additional requests, feel free to let me know!
- 13 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [63]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="litellm/bedrock/us.amazon.nova-pro-v1:0")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="litellm/bedrock/us.amazon.nova-pro-v1:0")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [64]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    # Get credentials and clean them (remove whitespace and quotes)
    access_key = os.environ.get('AWS_ACCESS_KEY_ID', '').strip().strip('"').strip("'")
    secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY', '').strip().strip('"').strip("'")
    session_token = os.environ.get('AWS_SESSION_TOKEN', '').strip().strip('"').strip("'")
    region = os.environ.get('AWS_REGION', 'us-east-1').strip().strip('"').strip("'")
    
    # Create SES client with cleaned credentials
    ses_client = boto3.client(
            'ses',
            region_name=region,
            aws_access_key_id=access_key if access_key else None,
            aws_secret_access_key=secret_key if secret_key else None,
            aws_session_token=session_token if session_token else None
        )
    
    try:
        response = ses_client.send_email(
            Source="greg.jacobs@thrivent.com",
            Destination={'ToAddresses': ["greg.jacobs@thrivent.com"]},
            Message={
                'Subject': {'Data': subject, 'Charset': 'UTF-8'},
                'Body': {'Html': {'Data': html_body, 'Charset': 'UTF-8'}}
            }
        )
        return {"status": "success", "message_id": response['MessageId']}
    except ClientError as e:
        return {"status": "error", "message": str(e)}

In [65]:
tools = [subject_tool, html_tool, send_html_email]

In [66]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114a8e340>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110bfc2c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 Function

In [67]:
instructions = """You are an email formatter and sender. 
When you receive a message containing an email body, you should:
1. Extract the email body content from the message
2. Use the subject_writer tool to write a subject for the email
3. Use the html_converter tool to convert the body to HTML
4. Use the send_html_email tool to send the email with the subject and HTML body

If the message contains the email body, proceed immediately with these steps. 
Do not ask for the email body if it was provided in the handoff context."""


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="litellm/bedrock/us.amazon.nova-pro-v1:0",
    handoff_description="Convert an email to HTML and send it")

### Now we have 3 tools and 1 handoff

In [68]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x114a8fa60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ecd6520>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', description='Write a 

In [69]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Output and Handoff: First, output a message with the complete email body text you selected. 
   Then immediately hand off to the 'Email Manager' agent.
   
   IMPORTANT: You MUST output the full email body in your message BEFORE the handoff so the Email Manager can see it.
   Say something like: "I've selected the best email. Here is the complete email body to send:
   
   [PASTE THE COMPLETE EMAIL HERE]
   
   I'm now handing off to the Email Manager to format and send this email."
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must output the complete email body in your message BEFORE calling the handoff.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="litellm/bedrock/us.amazon.nova-pro-v1:0")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [71]:
# Check if email was sent!
print("=== EMAIL SENDING CHECK ===\n")

# Look for send_html_email tool call result
for i, item in enumerate(result.new_items):
    item_type = type(item).__name__
    
    # Find the send_html_email tool call and its output
    if item_type == 'ToolCallItem' and hasattr(item, 'tool_calls'):
        for tc in item.tool_calls:
            if tc.function.name == 'send_html_email':
                print(f"Item {i}: send_html_email was called!")
                print(f"Arguments: {tc.function.arguments[:200]}...")
    
    # Find the output from send_html_email
    if item_type == 'ToolCallOutputItem' and hasattr(item, 'output'):
        output_str = str(item.output)
        if 'message_id' in output_str or 'success' in output_str:
            print(f"\nItem {i}: send_html_email OUTPUT:")
            print(output_str)

print(f"\n\nFinal Result:")
print(result.final_output)

=== EMAIL SENDING CHECK ===


Item 19: send_html_email OUTPUT:
{'status': 'success', 'message_id': '010f019c3dd6a00d-d192711f-b9f6-4837-9cc7-8819f7b27650-000000'}


Final Result:
The email has been successfully sent. Here are the details:

- **Status:** Success
- **Message ID:** 010f019c3dd6a00d-d192711f-b9f6-4837-9cc7-8819f7b27650-000000

If you need any further assistance, feel free to ask!


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.